In [ ]:
import os
import torch
from torch import nn
from torch import optim
import torchvision as tv
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


class Generator(nn.Module):
    def __init__(self, latent_dim=100, batchnorm=True):
        """A generator for mapping a latent space to a sample space.
        The sample space for this generator is single-channel, 28x28 images
        with pixel intensity ranging from -1 to +1.
        Args:
            latent_dim (int): latent dimension ("noise vector")
            batchnorm (bool): Whether or not to use batch normalization
        """
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.batchnorm = batchnorm
        self._init_modules()

    def _init_modules(self):
        """Initialize the modules."""
        # Project the input
        self.linear1 = nn.Linear(self.latent_dim, 136*6*5, bias=False)
        self.bn1d1 = nn.BatchNorm1d(136*6*5)
        self.leaky_relu = nn.LeakyReLU()

        # Convolutions
#         self.conv1 = nn.Conv2d(
#                 in_channels=136,
#                 out_channels=68,
#                 kernel_size=3,
#                 stride=1,
#                 padding=1,
#                 bias=False)
        #6X5
        self.bn2d1 = nn.BatchNorm2d(136) if self.batchnorm else None

        self.conv2 = nn.ConvTranspose2d(
                in_channels=136,
                out_channels=68,
                kernel_size=2,
                stride=1,
                padding=0,
                bias=False)
        #7X6
        self.bn2d2 = nn.BatchNorm2d(68) if self.batchnorm else None

        self.conv3 = nn.ConvTranspose2d(
                in_channels=68,
                out_channels=17,
                kernel_size=3,
                stride=2,
                padding=1,
                bias=False)
        #13X11
        self.tanh = nn.Tanh()

    def forward(self, input_tensor):
        """Forward pass; map latent vectors to samples."""
        intermediate = self.linear1(input_tensor)
#         intermediate = self.bn1d1(intermediate)
#         intermediate = self.leaky_relu(intermediate)

        intermediate = intermediate.view((-1, 136, 6, 5))

        #intermediate = self.conv1(intermediate)
        if self.batchnorm:
            intermediate = self.bn2d1(intermediate)
        intermediate = self.leaky_relu(intermediate)

        intermediate = self.conv2(intermediate)
        
        #print(intermediate.size())
        if self.batchnorm:
            intermediate = self.bn2d2(intermediate)
        intermediate = self.leaky_relu(intermediate)

        intermediate = self.conv3(intermediate)
        
        #print(intermediate.size())
        intermediate = intermediate.narrow(3, 0, 10)
        output_tensor = intermediate
        output_tensor = self.tanh(intermediate)
        

        
        return output_tensor


class Discriminator(nn.Module):
    def __init__(self):
        """A discriminator for discerning real from generated images.
        Images must be single-channel and 28x28 pixels.
        Output activation is Sigmoid.
        """
        super(Discriminator, self).__init__()
        self._init_modules()

    def _init_modules(self):
        """Initialize the modules."""
        self.conv1 = nn.Conv2d(
                in_channels=17,
                out_channels=68,
                kernel_size=4,
                stride=2,
                padding=2,
                bias=False)
        #7X6
        self.leaky_relu = nn.LeakyReLU()
        self.dropout_2d = nn.Dropout2d(0.2)

        self.conv2 = nn.Conv2d(
                in_channels=68,
                out_channels=136,
                kernel_size=2,
                stride=1,
                padding=0,
                bias=False)
        #6X5
        self.linear1 = nn.Linear(136*6*5, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        """Forward pass; map samples to confidence they are real [0, 1]."""
        intermediate = self.conv1(input_tensor)
        
        #print(intermediate.size())
        intermediate = self.leaky_relu(intermediate)
        intermediate = self.dropout_2d(intermediate)

        intermediate = self.conv2(intermediate)
        
        #print(intermediate.size())
        intermediate = self.leaky_relu(intermediate)
        intermediate = self.dropout_2d(intermediate)

        intermediate = intermediate.view((-1, 136*6*5))
        intermediate = self.linear1(intermediate)
        output_tensor = self.sigmoid(intermediate)
        #print("in dis forward")
        #print(output_tensor.size())
        #print(output_tensor)
        #print("finished")
        return output_tensor

class DCGAN():
    def __init__(self, latent_dim, noise_fn, dataloader,
                 batch_size=32, device='cpu', lr_d=1e-4, lr_g=4e-3):
        """
        Args:
            generator: a Ganerator network
            discriminator: A Discriminator network
            noise_fn: function f(num: int) -> pytorch tensor, (latent vectors)
            dataloader: a pytorch dataloader for loading images
            batch_size: training batch size. Must match that of dataloader
            device: cpu or CUDA
            lr_d: learning rate for the discriminator
            lr_g: learning rate for the generator
        """
        self.generator = Generator(latent_dim).to(device)
        self.discriminator = Discriminator().to(device)
        self.noise_fn = noise_fn
        self.dataloader = dataloader
        self.batch_size = batch_size
        self.device = device
        self.criterion = nn.BCELoss()
        self.optim_d = optim.Adam(self.discriminator.parameters(),
                                  lr=lr_d, betas=(0.5, 0.999))
        self.optim_g = optim.Adam(self.generator.parameters(),
                                  lr=lr_g, betas=(0.5, 0.999))
        self.target_ones = torch.ones((batch_size, 1), device=device)
        self.target_zeros = torch.zeros((batch_size, 1), device=device)
        
        self.bn2d = nn.BatchNorm2d(17)

    def generate_samples(self, latent_vec=None, num=None):
        """Sample images from the generator.
        Images are returned as a 4D tensor of values between -1 and 1.
        Dimensions are (number, channels, height, width). Returns the tensor
        on cpu.
        Args:
            latent_vec: A pytorch latent vector or None
            num: The number of samples to generate if latent_vec is None
        If latent_vec and num are None then use self.batch_size
        random latent vectors.
        """
        num = self.batch_size if num is None else num
        latent_vec = self.noise_fn(num) if latent_vec is None else latent_vec
        with torch.no_grad():
            samples = self.generator(latent_vec)
        samples = samples.cpu()  # move images to cpu
        return samples

    def train_step_generator(self):
        """Train the generator one step and return the loss."""
        self.generator.zero_grad()

        latent_vec = self.noise_fn(self.batch_size)
        generated = self.generator(latent_vec)
        classifications = self.discriminator(generated)
        loss = self.criterion(classifications, self.target_ones)
        loss.backward()
        self.optim_g.step()
        return loss.item()

    def train_step_discriminator(self, real_samples):
        """Train the discriminator one step and return the losses."""
        self.discriminator.zero_grad()

        # real samples
        pred_real = self.discriminator(real_samples)
        #print("size of pre_real")
        #print(pred_real.size())
        #print("size of ones")
        #print(self.target_ones.size())
        loss_real = self.criterion(pred_real, self.target_ones)

        # generated samples
        latent_vec = self.noise_fn(self.batch_size)
        with torch.no_grad():
            fake_samples = self.generator(latent_vec)
        pred_fake = self.discriminator(fake_samples)
        loss_fake = self.criterion(pred_fake, self.target_zeros)

        # combine
        loss = (loss_real + loss_fake) / 2
        loss.backward()
        self.optim_d.step()
        return loss_real.item(), loss_fake.item()

    def train_epoch(self, print_frequency=10, max_steps=0):
        """Train both networks for one epoch and return the losses.
        Args:
            print_frequency (int): print stats every `print_frequency` steps.
            max_steps (int): End epoch after `max_steps` steps, or set to 0
                             to do the full epoch.
        """
        loss_g_running, loss_d_real_running, loss_d_fake_running = 0, 0, 0
        for batch, (real_samples) in enumerate(self.dataloader):
            #print("in train_epoch")
            #print(batch)
            #print(real_samples.size())
            #print()
            real_samples = real_samples.to(self.device)
#             real_samples = self.bn2d(real_samples)
            ldr_, ldf_ = self.train_step_discriminator(real_samples)
            loss_d_real_running += ldr_
            loss_d_fake_running += ldf_
           
            if max_steps and batch == max_steps:
                break
        for i in range(batch*2):
            loss_g_running += self.train_step_generator()
        if print_frequency:
            print()
        loss_g_running /= batch
        loss_d_real_running /= batch
        loss_d_fake_running /= batch
        print("loss_g_running " + str(loss_g_running))
        print("loss_d_real_running" + str(loss_d_real_running))
        print("loss_d_fake_running" + str(loss_d_fake_running))
        return (loss_g_running, (loss_d_real_running, loss_d_fake_running))
    
    def trans_samples(self, samples):
        room_list = []
        obj_list = []
        obj_dict = {"label":0,
                     "x":0,
                     "y":0,
                     "orientation":0}
        for num in range(samples.size(dim=0)):
            for i in range(samples.size(dim=1)-2):
                for j in range(samples.size(dim=2)):
                    for k in range(samples.size(dim=3)):
                        if samples[num][i][j][k] == 1:
                            obj_dict["label"] = i
                            obj_dict["y"] = j
                            obj_dict["x"] = k
                            if samples[num][16][j][k] == -1:
                                obj_dict["orientation"] = 0
                            elif samples[num][16][j][k] == 1:
                                obj_dict["orientation"] = 2
                            elif samples[num][15][j][k] == -1:
                                obj_dict["orientation"] = 1
                            else:
                                obj_dict["orientation"] = 3
                            #tmp = obj_dict.copy()
                            obj_list.append(obj_dict.copy())
            room_list.append(obj_list.copy())
            obj_list.clear()
                            
        return room_list
    
    def save(self):
        torch.save(self.discriminator.state_dict(), "saveD.pt")
        torch.save(self.generator.state_dict(), "saveG.pt")
        print("saved")

def main():
    import matplotlib.pyplot as plt
    from time import time
    import urllib, json
    import torch

    
    batch_size = 16
    epochs = 1000
    latent_dim = 16
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    
#     url = "https://raw.githubusercontent.com/Chaoyuuu/Gather-Town-Datasets/master/datasets-ugly-rooms.json"
#     response = urllib.request.urlopen(url)
#     data_ugly = json.loads(response.read())
    
    url = "https://raw.githubusercontent.com/Chaoyuuu/Gather-Town-Datasets/master/datasets-beautiful-rooms.json"
    response = urllib.request.urlopen(url)
    data_origin = json.loads(response.read())

#     data = data_ugly + data_origin
    data = data_origin
    input_data = torch.zeros(len(data)-(len(data)%batch_size), 17, 13, 10)
#     input_data = torch.full((len(data)-(len(data)%batch_size), 17, 13, 10), 0.1)
    
    
    for i in range(len(input_data)):
        for j in range(len(data[i]["room"])):
            
            if data[i]["room"][j]["x"] > 9:
                continue
            input_data[i][data[i]["room"][j]["label"]][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
            if data[i]["room"][j]["orientation"] == 0:
                input_data[i][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
            elif data[i]["room"][j]["orientation"] == 1:
                input_data[i][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
            elif data[i]["room"][j]["orientation"] == 2:
                input_data[i][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
            else:
                input_data[i][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1 
      
    
    
    dataloader = DataLoader(input_data,
            batch_size=batch_size,
            shuffle=True,
            num_workers=1
            )
    noise_fn = lambda x: torch.randn((x, latent_dim), device=device)
    gan = DCGAN(latent_dim, noise_fn, dataloader, device=device, batch_size=batch_size)
    start = time()
    stable = 0
    for i in range(epochs):
        print(f"Epoch {i+1}; Elapsed time = {int(time() - start)}s")
        lg, (ldr, ldf) = gan.train_epoch()
        if ldr < 0.6 and ldr > 0.5 and ldf < 0.6 and ldf >0.5:
            stable += 1
        else:
            stable = 0
        
        result_tensor = gan.generate_samples()
        for j in range(2):
            output_tensor = result_tensor[j]
            object_dict = {
            0: 'WB',
            1: 'PR',
            2: 'T2',
            3: 'TV',
            4: 'B4',
            5: 'PP',
            6: 'MB',
            7: 'CC',
            8: 'CS',
            9: 'T3',
            10: 'B2',
            11: 'LP',
            12: 'MP',
            13: 'LB',
            14: 'DC'
            }

            threshold = 0.7
            enough = False
            
            while not enough:
                threshold -= 0.05
                num = 0
                room = [['--']*10 for i in range(13)]
                for col in range(13):
                    for row in range(10):
                        object_duplicate_flage = 0
                        max_value = 0
                        max_index = -1
                        for depth in range(15):
                            data = output_tensor[depth][col][row]

                            if data >= threshold and max_value < data:
                                max_index = depth
                                max_value = data


                        if max_index != -1:
                            num += 1
                            room[col][row] = object_dict[max_index]
                if num > 10:
                    enough = True
            for col in range(13):
                print(room[col])
            print()
        if stable >= 5:
            gan.save()
            break
        

        
   

if __name__ == "__main__":
    main()

Epoch 1; Elapsed time = 0s

loss_g_running 1.0181788610560554
loss_d_real_running0.751551879303796
loss_d_fake_running0.7006177987371173
['PR', 'DC', 'TV', 'LP', 'B4', 'T2', 'TV', 'LB', 'TV', 'DC']
['PP', 'B4', 'T2', 'MB', 'T2', 'MB', 'T2', 'MB', 'T2', 'MB']
['PR', 'CC', 'B4', 'T3', 'B4', 'LP', 'T2', 'T2', 'T2', 'DC']
['PP', 'TV', 'T2', 'MB', 'T2', 'TV', 'T2', 'TV', 'T2', 'TV']
['PR', 'LB', 'T2', 'LP', 'T2', 'LP', 'T2', 'LP', 'T2', 'B4']
['PP', 'LB', 'T2', 'MB', 'T2', 'TV', 'T2', 'TV', 'T2', 'B4']
['B4', 'LB', 'T2', 'T2', 'B4', 'LP', 'T2', 'LP', 'T2', 'LP']
['PP', 'T2', 'T2', 'TV', 'T2', 'TV', 'T2', 'TV', 'T2', 'MB']
['PR', 'LB', 'T2', 'LB', 'T2', 'LP', 'T2', 'T3', 'T2', 'DC']
['PP', 'LB', 'T2', 'TV', 'T2', 'TV', 'T2', 'TV', 'T2', 'B4']
['B4', 'LB', 'T2', 'LP', 'T2', 'T3', 'B4', 'LP', 'T2', 'LP']
['PP', 'WB', 'WB', 'WB', 'WB', 'CS', 'T3', 'WB', 'WB', 'MB']
['T3', 'DC', 'T2', 'DC', 'LP', 'DC', 'PR', 'DC', 'PR', 'DC']

['B4', 'DC', 'TV', 'LP', 'B4', 'LP', 'TV', 'LB', 'TV', 'LP']
['PP', '


loss_g_running 1.5366400565419878
loss_d_real_running0.8722812746252332
loss_d_fake_running0.6692007780075073
['--', '--', '--', '--', 'B4', '--', 'B4', '--', 'B4', 'PP']
['--', '--', '--', '--', '--', '--', '--', 'MB', 'T2', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', 'T2', 'TV']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'MB']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', 'B4', '--', 'B4', '--', 'B4', 'T2', 'B4']

['--', '--', '--', '--', 'B4', '--', '--', '--', 'B4', 'T3']
['--', '--', '--', '--', '--', '--


loss_g_running 1.3974877638476235
loss_d_real_running0.8011476993560791
loss_d_fake_running0.7304684221744537
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', 'CS', 'LB', 'DC', '--', '--', '--', '--']
['--', '--', '--', '--', 'PP', '--', '--', '--', '--', '--']
['--', '--', '--', 'CS', '--', '--', '--', 'CS', '--', '--']
['--', '--', '--', '--', '--', '--', 'CS', 'CS', '--', '--']
['--', '--', '--', 'CS', '--', '--', '--', 'MB', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', 'T2', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', 'CS', '--', 'CS', '--', '--', '--', 'CS', 'PR']

['--', 'MB', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--


loss_g_running 1.3712974744183677
loss_d_real_running0.7758825038160596
loss_d_fake_running0.7399856192725045
['--', '--', 'LP', '--', '--', '--', '--', '--', '--', 'PP']
['CC', '--', 'CC', '--', 'CC', '--', 'CC', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', 'MB', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', 'CS', 'CS', 'TV', 'CC', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', 'TV', 'MB', '--', '--']
['--', '--', '--', '--', '--', '--', 'CC', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', 'MP', '--', '--', '--', '--', '--', '--', '--', '--']

['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--


loss_g_running 1.387080797127315
loss_d_real_running0.7733237573078701
loss_d_fake_running0.7422708400658199
['--', '--', '--', '--', '--', 'TV', '--', '--', '--', 'PP']
['--', '--', 'WB', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', 'B2', '--', '--']
['--', '--', 'WB', 'CS', '--', '--', '--', '--', '--', '--']
['--', '--', 'PR', '--', '--', '--', 'CS', 'DC', '--', '--']
['--', '--', '--', '--', 'T3', 'CC', '--', '--', 'MB', '--']
['--', '--', '--', '--', '--', '--', 'CS', 'DC', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', 'DC', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', 'PP', '--', '--']

['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--'

...


loss_g_running 1.3844827967030662
loss_d_real_running0.7583426237106323
loss_d_fake_running0.7449801947389331
['PP', 'PP', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['CC', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', 'PP', '--', '--', 'T3', '--', '--', '--']
['--', '--', '--', 'DC', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'B2']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'MP']

['PP', 'PP', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--


loss_g_running 1.3608571674142564
loss_d_real_running0.7565969611917224
loss_d_fake_running0.7478886544704437
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', 'T3', 'CS', '--', '--']
['--', '--', '--', '--', '--', 'PR', 'LP', 'B2', '--', '--']
['--', '--', '--', '--', '--', 'LP', '--', 'CS', '--', '--']
['--', '--', '--', '--', '--', 'PR', 'MP', 'DC', '--', '--']
['--', '--', '--', '--', '--', '--', '--', 'CS', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']

['B4', '--', '--', '--', 'WB', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--


loss_g_running 1.3879743473870414
loss_d_real_running0.7479325618062701
loss_d_fake_running0.7416750405515943
['LB', '--', '--', '--', 'TV', '--', '--', 'WB', 'PP', 'PP']
['--', '--', '--', 'CS', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', 'B4', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', 'CS', 'TV']
['--', '--', '--', 'CS', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', 'MB', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', 'B2', '--', '--', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']

['LB', '--', '--', 'MP', 'TV', 'MP', '--', 'WB', 'PP', 'PP']
['--', 'LP', '--', 'CS', 'MB', 'CS


loss_g_running 1.3539156828607832
loss_d_real_running0.7553694205624717
loss_d_fake_running0.7464675434998104
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', 'CC', '--', '--', '--', '--', '--', '--']
['--', 'CS', '--', 'CC', '--', '--', '--', '--', '--', '--']
['--', 'CS', 'WB', 'CC', '--', '--', '--', '--', '--', '--']
['--', 'CS', '--', 'CS', '--', '--', '--', '--', '--', '--']
['--', '--', 'PP', 'CC', '--', 'CC', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']

['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--


loss_g_running 1.3579557027135576
loss_d_real_running0.7589429191180638
loss_d_fake_running0.7489833193165916
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', 'MB', '--', '--', '--', '--']
['--', '--', '--', 'CS', '--', 'T3', '--', '--', '--', '--']
['--', '--', '--', 'B4', 'DC', 'B2', '--', '--', '--', '--']
['--', '--', '--', '--', 'WB', 'T2', '--', '--', '--', '--']
['--', '--', '--', 'LP', 'LB', 'PP', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']

['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['PP', '--', '--', '--', '--', '--